In [1]:
import sys
import datetime as dt

import pandas as pd
import numpy as np
from IPython.display import display, HTML

sys.path.append('../invoicing')
from main import *

%load_ext autoreload
%autoreload 2

data_dir = '../tests/data/'



In [3]:
path = data_dir + 'timesheet.csv'
timesheet = read_timesheet(path, input_time_units='min')
timesheet

,date,project,time_spent,comment
0,2016-04-01,A,4.000000,NaN
1,2016-04-04,B,2.000000,NaN
2,2016-04-04,C,1.083333,NaN
3,2016-04-06,A,10.000000,NaN
4,2016-04-13,A,5.333333,booyah
5,2016-04-14,D,12.233333,doh
6,2016-04-15,A,8.350000,NaN


In [4]:
compute_project_summary(timesheet, freq='W')


,start_date,end_date,project,time_spent
0,2016-03-28,2016-04-03,A,4.000000
1,2016-04-04,2016-04-10,A,10.000000
3,2016-04-04,2016-04-10,B,2.000000
4,2016-04-04,2016-04-10,C,1.083333
2,2016-04-11,2016-04-17,A,13.683333
5,2016-04-11,2016-04-17,D,12.233333


In [5]:
base_fee = 1

biller1 = build_linear_biller(rate=30, base_fee=base_fee, freq='M')

bins = [10, 5, 7, np.inf]
rates = [30, 60, 90, 180]
biller2 = build_piecewise_linear_biller(bins, rates, base_fee, freq='W')

decompose(17, bins)
biller2(17)

cs = compute_cost_summary(timesheet, biller2)
cs

,start_date,end_date,time_spent,rate,cost
0,2016-03-28,2016-04-03,0.000000,inf,1.0
1,2016-03-28,2016-04-03,4.000000,30.000000,120.0
5,2016-04-04,2016-04-10,0.000000,inf,1.0
6,2016-04-04,2016-04-10,10.000000,30.000000,300.0
7,2016-04-04,2016-04-10,3.083333,60.000000,185.0
10,2016-04-11,2016-04-17,0.000000,inf,1.0
11,2016-04-11,2016-04-17,10.000000,30.000000,300.0
12,2016-04-11,2016-04-17,5.000000,60.000000,300.0
13,2016-04-11,2016-04-17,7.000000,90.000000,630.0
14,2016-04-11,2016-04-17,3.916667,180.000000,705.0


In [96]:
context = {
  'invoice_code': '#my-first-invoice',
  'issue_date': '2016-04-30',
  'due_date': '2016-05-01',
  'sender': 'Bumptious Q. Bangwhistle',
  'sender_address': '1 Queen St, Auckland 1001',
  'sender_email': 'hello@bangwhistle.net',
  'sender_bank_name': 'Bingo Bank',
  'sender_bank_account': '1234-56-789',
  'receiver': 'Wingnut',
  'receiver_address': '2 Victoria St, Auckland 1001',
  }

template_dir = 'example/'
html = make_invoice(cs, template_dir + 'invoice.html', context)
output_path = '../invoicing/templates/' + template_dir + 'test.html'

with open(output_path, 'w') as tgt:
    tgt.write(html)
    